In [21]:
import numpy as np
import pandas as pd
import torch
from src.utils.padding import pad_sequence
import sklearn.preprocessing as p

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
nlub_path = '/home/rad/projects/id-sf/data/datasets/nlu-benchmark/nlub.csv'

df = pd.read_csv(nlub_path, sep=';')

type(df)

pandas.core.frame.DataFrame

In [12]:
df.head(10)

,Unnamed: 0,tokens,slot_tags,intent
0,0,wake me up at five am this week,O O O O time time date date,alarm_set
1,1,wake me up at nine am on friday,O O O O time time O date,alarm_set
2,2,set an alarm for two hours from now,O O O O time time time time,alarm_set
3,3,quiet,O,audio_volume_mute
4,4,stop,O,audio_volume_mute
5,5,pause for ten seconds,O O time time,audio_volume_mute
6,6,pink is all we need,color_type O O O O,iot_hue_lightchange
7,7,make the lighting bit more warm here,O O O O O color_type O,iot_hue_lightchange
8,8,please set the lighting suitable for reading,O O O O color_type color_type color_type,iot_hue_lightchange
9,9,turn the lights off please,O O O O O,iot_hue_lightoff


In [13]:

df['tokens'] = df.apply(lambda row: pad_sequence(row['tokens'], max_len=35, pad_value='<pad>'), axis=1)
df['slot_tags'] = df.apply(lambda row: pad_sequence(row['slot_tags'], max_len=35, pad_value='O'), axis=1)

df.head(10)

,Unnamed: 0,tokens,slot_tags,intent
0,0,<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad...,O O O O O O O O O O O O O O O O O time time da...,alarm_set
1,1,<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad...,O O O O O O O O O O O O O O O O O time time O ...,alarm_set
2,2,<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad...,O O O O O O O O O O O O O O O O O time time ti...,alarm_set
3,3,<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad...,O O O O O O O O O O O O O O O O O O O O O O O ...,audio_volume_mute
4,4,<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad...,O O O O O O O O O O O O O O O O O O O O O O O ...,audio_volume_mute
5,5,<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad...,O O O O O O O O O O O O O O O O O time time O ...,audio_volume_mute
6,6,<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad...,O O O O O O O O O O O O O O O color_type O O O...,iot_hue_lightchange
7,7,<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad...,O O O O O O O O O O O O O O O O O O O color_ty...,iot_hue_lightchange
8,8,<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad...,O O O O O O O O O O O O O O O O O O color_type...,iot_hue_lightchange
9,9,<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad...,O O O O O O O O O O O O O O O O O O O O O O O ...,iot_hue_lightoff


In [18]:
x = df['tokens']
x = np.array([np.array(el.split(), dtype=object) for el in x])
x = np.stack(x)
x

array([['<pad>', '<pad>', '<pad>', ..., '<pad>', '<pad>', '<pad>'],
       ['<pad>', '<pad>', '<pad>', ..., '<pad>', '<pad>', '<pad>'],
       ['<pad>', '<pad>', '<pad>', ..., '<pad>', '<pad>', '<pad>'],
       ...,
       ['<pad>', '<pad>', '<pad>', ..., '<pad>', '<pad>', '<pad>'],
       ['<pad>', '<pad>', '<pad>', ..., '<pad>', '<pad>', '<pad>'],
       ['<pad>', '<pad>', '<pad>', ..., '<pad>', '<pad>', '<pad>']],
      dtype=object)

In [25]:
slots = df['slot_tags']
slots = np.array([np.array(el.split(), dtype=object) for el in slots])
slots = np.stack(slots)

# (n_samples * max_seq_len)
shape = slots.shape
slots_flat = slots.flatten()

unique, inverse = np.unique(slots_flat, return_inverse=True)
onehot = np.eye(unique.shape[0])[inverse]

onehot_shape = shape + (-1,)

slots = onehot.reshape(onehot_shape)

slots

array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

In [31]:
intents = np.array(df['intent']).reshape(-1, 1)

i = torch.Tensor(p.OneHotEncoder().fit_transform(intents).toarray())
i.shape

torch.Size([25701, 68])